In [1]:
# ! pip install deepctr==0.8.5 --no-deps
# ! pip install torch==1.7.0 torchvision==0.8.1 
# ! pip install tensorflow-gpu==1.13.1
# ! pip install numba

In [2]:
import sys
sys.path.append('..')
sys.path.append('../../config/')
from config_prosper import *
import os
import gc
import pandas as pd
import numpy as np
import tensorflow as tf

from time import time
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names,VarLenSparseFeat
from mytools.utils.myfile import savePkl,loadPkl
from mmoe_tf import MMOE
from evaluation import evaluate_deepctr
from tensorflow.python.keras.utils import multi_gpu_model
from tqdm import tqdm as tqdm

BASE_DIR(目录): /home/tione/notebook


In [3]:
# GPU相关设置
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
def loadFeedinfo():
    feed = loadPkl(FEED_INFO_DEAL)
    feed[["bgm_song_id", "bgm_singer_id"]] += 1  # 0 用于填未知
    feed[["bgm_song_id", "bgm_singer_id", "videoplayseconds"]] = \
        feed[["bgm_song_id", "bgm_singer_id", "videoplayseconds"]].fillna(0)
    feed['bgm_song_id'] = feed['bgm_song_id'].astype('int64')
    feed['bgm_singer_id'] = feed['bgm_singer_id'].astype('int64')
    print('feedinfo loading over...')
    return feed

def getFeedembeddings(df):
    #feedembeddings 降维

    feed_embedding_path = os.path.join(FEATURE_PATH,'feedembedings.pkl')
    feed_embeddings = loadPkl(feed_embedding_path)
    df = df.merge(feed_embeddings,on='feedid',how='left')
    dense = [x for x in list(feed_embeddings.columns) if x != 'feedid' ]
    
    return df,dense

def getSvdembeddings(df):
    dense = []
    #userid-feedid svd
    svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_feedid_embedding.pkl'))
    df = df.merge(svd_embedding,on = ['userid'],how='left')
    dense += [x for x in list(svd_embedding.columns) if x not in ['userid']]
                            
    #userid_authorid svd
    svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_authorid_embedding.pkl'))
    df  = df.merge(svd_embedding,on = ['userid'],how='left')
    dense += [x for x in list(svd_embedding.columns) if x not in ['userid']]
    
    #text svd
    svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'texts_svd_embedding.pkl'))
    svd_embedding['feedid']  = svd_embedding['feedid'].astype(np.int32) 
    df  = df.merge(svd_embedding,on = ['feedid'],how='left')
    dense += [x for x in list(svd_embedding.columns) if x not in ['feedid']]
    
    return df, dense
def myLeftjoin(left,right,on):
    return left.merge(right[right[on].isin(left[on])].set_index(on),how='left',left_on=on,right_index=True)
def getHistFeatures(df,hist_features):
    dense = [x for x in hist_features.columns if x not in df.columns and  'hist_seq' not in x ]
    varlen = [x for x in hist_features.columns if 'hist_seq' in x]
    df = df.merge(hist_features[hist_features.userid.isin(df.userid.unique())][['userid','feedid','date_','device'] + dense],how = 'left',on = ['userid','feedid','date_','device'])
    return (df,dense)

In [5]:
# -*- coding:utf-8 -*-
"""
Author:
    Yiyuan Liu, lyy930905@gmail.com
    zanshuxun, zanshuxun@aliyun.com
Reference:
    [1] [Jiaqi Ma, Zhe Zhao, Xinyang Yi, et al. Modeling Task Relationships in Multi-task Learning with Multi-gate Mixture-of-Experts[C]](https://dl.acm.org/doi/10.1145/3219819.3220007)
"""
import tensorflow as tf

from deepctr.feature_column import build_input_features, get_linear_logit, DEFAULT_GROUP_NAME, input_from_feature_columns
from deepctr.layers.utils import concat_func, add_func, combined_dnn_input
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM
from tensorflow.python.keras.initializers import glorot_normal
from tensorflow.python.keras.layers import Layer


class MMOELayer(Layer):
    """
    The Multi-gate Mixture-of-Experts layer in MMOE model
      Input shape
        - 2D tensor with shape: ``(batch_size,units)``.  
      Output shape
        - A list with **num_tasks** elements, which is a 2D tensor with shape: ``(batch_size, output_dim)`` .
      Arguments
        - **num_tasks**: integer, the number of tasks, equal to the number of outputs.
        - **num_experts**: integer, the number of experts.
        - **output_dim**: integer, the dimension of each output of MMOELayer.
    References
      - [Jiaqi Ma, Zhe Zhao, Xinyang Yi, et al. Modeling Task Relationships in Multi-task Learning with Multi-gate Mixture-of-Experts[C]](https://dl.acm.org/doi/10.1145/3219819.3220007)
    """

    def __init__(self, num_tasks, num_experts, output_dim, seed=1024, **kwargs):
        self.num_experts = num_experts
        self.num_tasks = num_tasks
        self.output_dim = output_dim
        self.seed = seed
        super(MMOELayer, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        self.expert_kernel = self.add_weight(
            name='expert_kernel',
            shape=(input_dim, self.num_experts * self.output_dim),
            dtype=tf.float32,
            initializer=glorot_normal(seed=self.seed))
        self.gate_kernels = []
        for i in range(self.num_tasks):
            self.gate_kernels.append(self.add_weight(
                name='gate_weight_'.format(i),
                shape=(input_dim, self.num_experts),
                dtype=tf.float32,
                initializer=glorot_normal(seed=self.seed)))
        super(MMOELayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        outputs = []
        expert_out = tf.tensordot(inputs, self.expert_kernel, axes=(-1, 0))
        expert_out = tf.reshape(expert_out, [-1, self.output_dim, self.num_experts])
        for i in range(self.num_tasks):
            gate_out = tf.tensordot(inputs, self.gate_kernels[i], axes=(-1, 0))
            gate_out = tf.nn.softmax(gate_out)
            gate_out = tf.tile(tf.expand_dims(gate_out, axis=1), [1, self.output_dim, 1])
            output = tf.reduce_sum(tf.multiply(expert_out, gate_out), axis=2)
            outputs.append(output)
        return outputs

    def get_config(self):

        config = {'num_tasks': self.num_tasks,
                  'num_experts': self.num_experts,
                  'output_dim': self.output_dim}
        base_config = super(MMOELayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return [input_shape[0], self.output_dim] * self.num_tasks


def MMOE_FM(dnn_feature_columns, num_tasks, task_types, task_names, num_experts=4, expert_dim=8,l2_reg_linear=1e-5,dnn_hidden_units=(128, 128),tower_dnn_units_lists=[[64,32,8],[64,32]],
         l2_reg_embedding=1e-5, l2_reg_dnn=0, seed=1024, dnn_dropout=0, dnn_activation='relu',dnn_use_bn = True,fm_group=[DEFAULT_GROUP_NAME]):
    """Instantiates the Multi-gate Mixture-of-Experts architecture.
    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param num_tasks: integer, number of tasks, equal to number of outputs, must be greater than 1.
    :param tasks: list of str, indicating the loss of each tasks, ``"binary"`` for  binary logloss, ``"regression"`` for regression loss. e.g. ['binary', 'regression']
    :param num_experts: integer, number of experts.
    :param expert_dim: integer, the hidden units of each expert.
    :param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of shared-bottom DNN
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param l2_reg_dnn: float. L2 regularizer strength applied to DNN
    :param task_dnn_units: list,list of positive integer or empty list, the layer number and units in each layer of task-specific DNN
    :param seed: integer ,to use as random seed.
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param dnn_activation: Activation function to use in DNN
    :return: a Keras model instance
    """
    if num_tasks <= 1:
        raise ValueError("num_tasks must be greater than 1")
    if len(task_types) != num_tasks:
        raise ValueError("num_tasks must be equal to the length of tasks")
    for task_type in task_types:
        if task_type not in ['binary', 'regression']:
            raise ValueError("task must be binary or regression, {} is illegal".format(task_type))
    if num_tasks != len(tower_dnn_units_lists):
        raise ValueError("the length of tower_dnn_units_lists must be euqal to num_tasks")
    
    features = build_input_features(dnn_feature_columns)

    inputs_list = list(features.values())
    
    linear_logit = get_linear_logit(features, [x for x in dnn_feature_columns if isinstance(x,DenseFeat)], seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear)

    group_embedding_dict, dense_value_list = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding,
                                                                        seed, support_group=True)

    fm_logit = add_func([FM()(concat_func(v, axis=1))
                         for k, v in group_embedding_dict.items() if k in fm_group])
    



    sparse_embedding_list, dense_value_list = input_from_feature_columns(features, dnn_feature_columns,
                                                                         l2_reg_embedding, seed)
    dnn_input = combined_dnn_input(sparse_embedding_list, dense_value_list)
    dnn_out = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                  False, seed=seed)(dnn_input)
    mmoe_outs = MMOELayer(num_tasks, num_experts, expert_dim)(dnn_out)
    
    task_outputs = []
    for task_type, task_name, tower_dnn, mmoe_out in zip(task_types, task_names, tower_dnn_units_lists, mmoe_outs):
        #build tower layer
        tower_output = DNN(tower_dnn, dnn_activation, l2_reg_dnn, dnn_dropout, dnn_use_bn, seed=seed, name='tower_'+task_name)(mmoe_out)
        
        logit = tf.keras.layers.Dense(1, use_bias=False, activation=None)(mmoe_out)
        final_logit = add_func([linear_logit, fm_logit, logit])
        output = PredictionLayer(task_type, name=task_name)(logit) 
        task_outputs.append(output)


    model = tf.keras.models.Model(inputs=inputs_list,
                                  outputs=task_outputs)
    return model



In [6]:
class myDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data: pd.DataFrame,feedinfo,dnn_feature_columns,batch_size=2048, shuffle=True):
        self.data = data.copy()
        self.target = ACTION_LIST
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(self.data.shape[0])
        
        self.feedinfo = feedinfo
#         self.feed_embeddings = loadPkl(os.path.join(FEATURE_PATH,'feedembedings.pkl'))
#         self.user_feed_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_feedid_embedding.pkl'))
#         self.user_author_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_authorid_embedding.pkl'))
#         self.text_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'texts_svd_embedding.pkl'))
#         self.text_svd_embedding['feedid'] = self.text_svd_embedding['feedid'].astype(int)



        self.graph_emb8 = loadPkl(os.path.join(MODEL_PATH,'emb/graph_walk_emb_8.pkl'))
        self.feed_emb_16 = loadPkl(os.path.join(MODEL_PATH,'emb/feed_embeddings_16.pkl'))
        self.weight_emb8 = loadPkl(os.path.join(MODEL_PATH,'emb/user_weight_emd_8.pkl'))
        self.weight_emb8 = self.weight_emb8.drop('user_date_weight_emd',axis = 1)
        self.keyword_w2v_8 = loadPkl(os.path.join(MODEL_PATH,'emb/keyword_w2v_8.pkl'))
        self.userid_feedid_d2v_all_16 = loadPkl(os.path.join(MODEL_PATH,'emb/userid_feedid_d2v_all_16.pkl'))##加了初赛数据
        self.all_text_data_v8 = loadPkl(os.path.join(MODEL_PATH,'emb/all_text_data_v8.pkl'))
        self.userid_authorid_d2v_all_16 = loadPkl(os.path.join(MODEL_PATH,'emb/userid_authorid_d2v_all_16.pkl'))
        
        self.dnn_feature_columns = dnn_feature_columns
        self.feature_names = get_feature_names(self.dnn_feature_columns)
        
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)

    def __len__(self):

        return (self.data.shape[0] // self.batch_size) + 1

    def __getitem__(self, index):
        batch_indexs = self.indexes[index * self.batch_size:(index + 1) *
                                    self.batch_size]
        batch_data = self.data.iloc[batch_indexs, :]
        
        return self.get_feature_on_batch(batch_data)

    def on_epoch_end(self):
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)
    def on_epoch_begain(self):
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)

    def get_feature_on_batch(self, batch):
        
#         batch = batch.merge(self.user_feed_svd_embedding,on='userid',how='left')
#         batch = batch.merge(self.user_author_svd_embedding,on='userid',how='left')
#         batch = batch.merge(self.text_svd_embedding,on='feedid',how='left')
#         batch = batch.merge(self.feed_embeddings,on='feedid',how='left')
                
        batch = batch.merge(self.graph_emb8, how='left',
              on='userid')
        batch = batch.merge(self.feed_emb_16, how='left',
                      on='feedid')
        batch = batch.merge(self.weight_emb8, how='left',
                      on='userid')
        batch = batch.merge(self.keyword_w2v_8, how='left',
                      on='feedid')
        batch = batch.merge(self.userid_feedid_d2v_all_16, how='left',
                      on='userid')
        batch = batch.merge(self.all_text_data_v8, how='left',
                      on='feedid')
        batch = batch.merge(self.userid_authorid_d2v_all_16, how='left',
                      on='userid')
                                                      
        x = {name: batch[name].values for name in self.feature_names}
        for col in ['manual_tag_list','manual_keyword_list','machine_keyword_list']:
            x[col] = np.array(batch[col].tolist())
        y = [batch[y].values for y in ACTION_LIST]
        
        return x,y

In [7]:
DEBUG = False
data = loadPkl(USER_ACTION)
data = data.head(1000000) if DEBUG else data
feedinfo = loadFeedinfo()
# feed_embeddings = loadPkl(os.path.join(FEATURE_PATH,'feedembedings.pkl'))
# user_feed_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_feedid_embedding.pkl'))
# user_author_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_authorid_embedding.pkl'))
# text_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'texts_svd_embedding.pkl'))

graph_emb8 = loadPkl(os.path.join(MODEL_PATH,'emb/graph_walk_emb_8.pkl'))
feed_emb_16 = loadPkl(os.path.join(MODEL_PATH,'emb/feed_embeddings_16.pkl'))
weight_emb8 = loadPkl(os.path.join(MODEL_PATH,'emb/user_weight_emd_8.pkl'))
weight_emb8 = weight_emb8.drop('user_date_weight_emd',axis = 1)
keyword_w2v_8 = loadPkl(os.path.join(MODEL_PATH,'emb/keyword_w2v_8.pkl'))
userid_feedid_d2v_all_16 = loadPkl(os.path.join(MODEL_PATH,'emb/userid_feedid_d2v_all_16.pkl'))##加了初赛数据
all_text_data_v8 = loadPkl(os.path.join(MODEL_PATH,'emb/all_text_data_v8.pkl'))
userid_authorid_d2v_all_16 = loadPkl(os.path.join(MODEL_PATH,'emb/userid_authorid_d2v_all_16.pkl'))


embedding_dim = 16
sparse_features = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id' ]
dense_features = ['videoplayseconds',]
for df in [
    graph_emb8,
    feed_emb_16,
    weight_emb8,
    keyword_w2v_8,
    userid_feedid_d2v_all_16,
    all_text_data_v8,
    userid_authorid_d2v_all_16
]:
    dense_features += [x for x in df.columns if x not in ['userid','feedid']]
    del df
    gc.collect()
    
    
data = data.merge(feedinfo[[
    'feedid', 'authorid', 'videoplayseconds', 'bgm_song_id',
    'bgm_singer_id'
] + ['manual_tag_list', 'manual_keyword_list', 'machine_keyword_list'
     ]],
                    how='left',
                    on='feedid')

#dense 特征处理
data['videoplayseconds'] = data['videoplayseconds'].fillna(0, )
data['videoplayseconds'] = np.log(data['videoplayseconds'] + 1.0)
train = data[data.date_ != 14]
val = data[data.date_==14]

feedinfo loading over...


In [8]:
fixlen_feature_columns = [
    SparseFeat(feat,
               vocabulary_size = feedinfo[feat].max() + 1,
               embedding_dim=embedding_dim) for feat in sparse_features if feat !='userid'
] + [DenseFeat(feat, 1) for feat in dense_features
] + [SparseFeat('userid',
               vocabulary_size= data['userid'].max() + 1,
               embedding_dim=embedding_dim)]
tag_columns = [
    VarLenSparseFeat(SparseFeat('manual_tag_list',
                                vocabulary_size=TAG_MAX,
                                embedding_dim=16),
                     maxlen=4)
]
key_words_columns = [
    VarLenSparseFeat(SparseFeat('manual_keyword_list',
                                vocabulary_size=KEY_WORDS_MAX,
                                embedding_dim=16),
                     maxlen=4),
    VarLenSparseFeat(SparseFeat('machine_keyword_list',
                                vocabulary_size=KEY_WORDS_MAX,
                                embedding_dim=16),
                     maxlen=4),
]
dnn_feature_columns =  fixlen_feature_columns + tag_columns + key_words_columns

In [10]:
num_tasks = len(ACTION_LIST)
train_model = MMOE_FM(dnn_feature_columns=dnn_feature_columns,
                    num_tasks=num_tasks,
                   task_types = ['binary' for i in range(num_tasks)],
                   task_names = ACTION_LIST,
                   num_experts=5,
                   tower_dnn_units_lists = [[64,32] for i in range(num_tasks) ],
                   dnn_hidden_units=(512, 512),
                   expert_dim=32,)
# train_model.summary()
train_loader = myDataGenerator(train,feedinfo,dnn_feature_columns,batch_size=4096)
val_loader = myDataGenerator(val,feedinfo,dnn_feature_columns,batch_size=4096 * 4,shuffle = False) # shuffle 必须为False
# len(train_loader)
# train_model = multi_gpu_model(train_model, gpus=2)
optimizer = tf.keras.optimizers.Adagrad(
    lr=0.05, epsilon=1e-07,
)
train_model.compile('adagrad', loss='binary_crossentropy')

shuffle data index ing...


## offline

In [11]:
best_score = -1
early_stop = 1
no_imporove = 0
for epoch in range(6):
    history = train_model.fit(train_loader,
                              epochs=1, verbose=1,workers = 8,use_multiprocessing=True,max_queue_size=200)
    pred_ans = train_model.predict_generator(val_loader)
    pred_ans = np.concatenate(pred_ans,1)
    pred_ans = pd.DataFrame(pred_ans,columns=ACTION_LIST)
    weightauc,uaucs = evaluate_deepctr(val_loader.data[ACTION_LIST],pred_ans,val_loader.data['userid'].values,ACTION_LIST)
    
    if best_score < weightauc:
        best_score = weightauc
        train_model.save_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_fm'))
        no_imporove = 0    
    else :
        no_imporove += 1
    if no_imporove >= early_stop:
        print('-----stoped on epoch %s ------- ' % (epoch))
        break
    

Instructions for updating:
Use tf.cast instead.
Please check the latest version manually on https://pypi.org/project/deepctr/#history
18013/18013 [==============================] - 1071s 59ms/step - loss: 0.2646 - read_comment_loss: 0.0911 - like_loss: 0.0909 - click_avatar_loss: 0.0362 - forward_loss: 0.0205 - comment_loss: 0.0033 - follow_loss: 0.0051 - favorite_loss: 0.0074
【UAUC：0.672448718177448】 [0.6460611687204344, 0.6359439362013574, 0.732645105154568, 0.7121590549091988, 0.6016575449796839, 0.7191566742659992, 0.7514933683569961]

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
18013/18013 [==============================] - 1065s 59ms/step - loss: 0.2460 - read_comment_loss: 0.0859 - like_loss: 0.0867 - click_avatar_loss: 0.0337 - forward_loss: 0.0189 - comment_loss: 0.0030 - follow_loss: 0.0046 - favorite_loss: 0.0061
【UAUC：0.67693398084457

In [12]:
early_stop = 1
no_imporove = 0
for epoch in range(6):
    history = train_model.fit(train_loader,
                              epochs=1, verbose=1,workers = 8,use_multiprocessing=True,max_queue_size=200)
    pred_ans = train_model.predict_generator(val_loader)
    pred_ans = np.concatenate(pred_ans,1)
    pred_ans = pd.DataFrame(pred_ans,columns=ACTION_LIST)
    weightauc,uaucs = evaluate_deepctr(val_loader.data[ACTION_LIST],pred_ans,val_loader.data['userid'].values,ACTION_LIST)
    
    if best_score < weightauc:
        best_score = weightauc
        train_model.save_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_fm'))
        no_imporove = 0    
    else :
        no_imporove += 1
    if no_imporove >= early_stop:
        print('-----stoped on epoch %s ------- ' % (epoch))
        break
    

18013/18013 [==============================] - 1099s 61ms/step - loss: 0.2343 - read_comment_loss: 0.0837 - like_loss: 0.0842 - click_avatar_loss: 0.0320 - forward_loss: 0.0172 - comment_loss: 0.0026 - follow_loss: 0.0042 - favorite_loss: 0.0054
【UAUC：0.6803212825055626】 [0.6485266048232415, 0.6430980601213966, 0.7381872971136003, 0.7246972518051261, 0.6246560154047662, 0.7297619348935097, 0.7652862765845532]

Consider using a TensorFlow optimizer from `tf.train`.
18013/18013 [==============================] - 1090s 61ms/step - loss: 0.2330 - read_comment_loss: 0.0835 - like_loss: 0.0839 - click_avatar_loss: 0.0318 - forward_loss: 0.0170 - comment_loss: 0.0026 - follow_loss: 0.0042 - favorite_loss: 0.0053
【UAUC：0.6802493044435942】 [0.6468564449705851, 0.6437740037169176, 0.7386339402430026, 0.7258464140951686, 0.6245010843327642, 0.7315671139287018, 0.7653106738909905]
-----stoped on epoch 1 ------- 


In [ ]:
print('over')

WARNING:tensorflow:From /opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
  160/16375 [..............................] - ETA: 38:00 - loss: 0.4543 - read_comment_loss: 0.2009 - like_loss: 0.1360 - click_avatar_loss: 0.0490 - forward_loss: 0.0303 - comment_loss: 0.0095 - follow_loss: 0.0101 - favorite_loss: 0.0160Please check the latest version manually on https://pypi.org/project/deepctr/#history
16374/16375 [============================>.] - ETA: 0s - loss: 0.2687 - read_comment_loss: 0.0916 - like_loss: 0.0924 - click_avatar_loss: 0.0373 - forward_loss: 0.0211 - comment_loss: 0.0035 - follow_loss: 0.0053 - favorite_loss: 0.0081shuffle data index ing...
16375/16375 [==============================] - 1503s 92ms/step - loss: 0.2687 - read_comment_loss: 0.0916 - like_loss: 0.0924 - click_avatar_loss: 0.0373 - forward_loss: 0.0211 - comment_loss: 0.0035 - follow_loss: 0.0053 - favorite_loss: 0.0081
【UAUC：0.6700101889732343】 [0.6442668842341969, 0.6336023812919299, 0.7284933881607184, 0.703039308704943, 0.6044826742324346, 0.7139985851427194, 0.7537504314379345]
WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adagrad object at 0x7ff4cc42d160>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
WARNING:tensorflow:From /opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
16374/16375 [============================>.] - ETA: 0s - loss: 0.2471 - read_comment_loss: 0.0856 - like_loss: 0.0869 - click_avatar_loss: 0.0341 - forward_loss: 0.0191 - comment_loss: 0.0031 - follow_loss: 0.0047 - favorite_loss: 0.0065shuffle data index ing...
16375/16375 [==============================] - 1525s 93ms/step - loss: 0.2471 - read_comment_loss: 0.0856 - like_loss: 0.0869 - click_avatar_loss: 0.0341 - forward_loss: 0.0191 - comment_loss: 0.0031 - follow_loss: 0.0047 - favorite_loss: 0.0065
【UAUC：0.6755101293690419】 [0.6485964198992606, 0.6374567272371356, 0.7343447150731571, 0.7134678791456773, 0.6082287812685985, 0.7243831381039088, 0.7601065918245948]
WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adagrad object at 0x7ff4cc42d160>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
16374/16375 [============================>.] - ETA: 0s - loss: 0.2420 - read_comment_loss: 0.0848 - like_loss: 0.0860 - click_avatar_loss: 0.0333 - forward_loss: 0.0184 - comment_loss: 0.0030 - follow_loss: 0.0045 - favorite_loss: 0.0060shuffle data index ing...
16375/16375 [==============================] - 1498s 91ms/step - loss: 0.2420 - read_comment_loss: 0.0848 - like_loss: 0.0860 - click_avatar_loss: 0.0333 - forward_loss: 0.0184 - comment_loss: 0.0030 - follow_loss: 0.0045 - favorite_loss: 0.0060
【UAUC：0.6768239834706983】 [0.6499220576729309, 0.6380985861773546, 0.7362522134729649, 0.7143876053885354, 0.6115306736212887, 0.7267547126851568, 0.7595503772543808]
WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adagrad object at 0x7ff4cc42d160>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
16374/16375 [============================>.] - ETA: 0s - loss: 0.2394 - read_comment_loss: 0.0843 - like_loss: 0.0855 - click_avatar_loss: 0.0329 - forward_loss: 0.0181 - comment_loss: 0.0029 - follow_loss: 0.0044 - favorite_loss: 0.0058shuffle data index ing...
16375/16375 [==============================] - 1496s 91ms/step - loss: 0.2394 - read_comment_loss: 0.0843 - like_loss: 0.0855 - click_avatar_loss: 0.0329 - forward_loss: 0.0181 - comment_loss: 0.0029 - follow_loss: 0.0044 - favorite_loss: 0.0058
【UAUC：0.6787680773661907】 [0.6504894177654035, 0.6398203406668407, 0.7376251435172757, 0.7191377159367602, 0.6175566966294476, 0.7280596857574646, 0.7625619273401175]
WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adagrad object at 0x7ff4cc42d160>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
16375/16375 [==============================] - 1502s 92ms/step - loss: 0.2378 - read_comment_loss: 0.0840 - like_loss: 0.0851 - click_avatar_loss: 0.0327 - forward_loss: 0.0179 - comment_loss: 0.0029 - follow_loss: 0.0044 - favorite_loss: 0.0057ata index ing...

【UAUC：0.6794855341450599】 [0.6503600652980819, 0.6404422585658697, 0.7373986097837447, 0.7218950130480467, 0.6193562622633948, 0.7307102318592538, 0.7637861802576577]
WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adagrad object at 0x7ff4cc42d160>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
16374/16375 [============================>.] - ETA: 0s - loss: 0.2365 - read_comment_loss: 0.0838 - like_loss: 0.0848 - click_avatar_loss: 0.0325 - forward_loss: 0.0178 - comment_loss: 0.0028 - follow_loss: 0.0043 - favorite_loss: 0.0056shuffle data index ing...
16375/16375 [==============================] - 1496s 91ms/step - loss: 0.2365 - read_comment_loss: 0.0838 - like_loss: 0.0848 - click_avatar_loss: 0.0325 - forward_loss: 0.0178 - comment_loss: 0.0028 - follow_loss: 0.0043 - favorite_loss: 0.0056
【UAUC：0.6791977646084176】 [0.6489407647286988, 0.6406420770663092, 0.7376803892598793, 0.7215399740620497, 0.6213240726725826, 0.7301762150034593, 0.7634806095378568]
-----stoped on epoch 5 ------- 
2
train_model.load_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_offline2'))

In [13]:
train_model.load_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_fm'))

# online

In [14]:
data_loader = myDataGenerator(data,feedinfo,dnn_feature_columns,batch_size=4096)
for epoch in range(1):
    history = train_model.fit(data_loader,
                              epochs=1, verbose=1,workers = 8,use_multiprocessing=True,max_queue_size=100)
    pred_ans = train_model.predict_generator(val_loader)
    pred_ans = np.concatenate(pred_ans,1)
    pred_ans = pd.DataFrame(pred_ans,columns=ACTION_LIST)
    weightauc,uaucs = evaluate_deepctr(val_loader.data[ACTION_LIST],pred_ans,val_loader.data['userid'].values,ACTION_LIST)
train_model.save_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_fm_online'))

shuffle data index ing...
19652/19652 [==============================] - 1169s 59ms/step - loss: 0.2333 - read_comment_loss: 0.0835 - like_loss: 0.0839 - click_avatar_loss: 0.0319 - forward_loss: 0.0170 - comment_loss: 0.0026 - follow_loss: 0.0042 - favorite_loss: 0.0053
【UAUC：0.7237923349941296】 [0.6929848871550134, 0.6760752262390787, 0.7752028403617471, 0.7833625569976088, 0.6896982934280934, 0.7853979660522914, 0.8002706303849083]

Consider using a TensorFlow optimizer from `tf.train`.


In [15]:
test = pd.read_csv('../../data/wedata/wechat_algo_data2/test_a.csv')
test = test.merge(feedinfo[['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id']+ ['manual_tag_list','manual_keyword_list','machine_keyword_list']], how='left',on='feedid')
test['videoplayseconds'] = test['videoplayseconds'].fillna(0, )
test['videoplayseconds'] = np.log(test['videoplayseconds'] + 1.0)
test[ACTION_LIST] = 0
t1 = time()
test_loader = myDataGenerator(test,feedinfo,dnn_feature_columns,shuffle=False,batch_size=4096*20)
pred_ans = train_model.predict(test_loader)
t2 = time()
print('7个目标行为%d条样本预测耗时（毫秒）：%.3f' % (len(test), (t2 - t1) * 1000.0))
ts = (t2 - t1) * 1000.0 / len(test) * 2000.0
print('7个目标行为2000条样本平均预测耗时（毫秒）：%.3f' % ts)

# 5.生成提交文件
for i, action in enumerate(ACTION_LIST):
    test[action] = pred_ans[i]
test[['userid', 'feedid'] + ACTION_LIST].to_csv(os.path.join(SUMIT_DIR,'tf_mmoe_fm.csv'), index=None, float_format='%.6f')
print('to_csv ok')

7个目标行为4252097条样本预测耗时（毫秒）：36428.298
7个目标行为2000条样本平均预测耗时（毫秒）：17.134
to_csv ok
